# Some experiments to understand the gates as matrix transformations

Reference: [Multi-qubit gates](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html#Multi-Qubit-Gates)

Start with some linear algebra...

In [56]:
import numpy as np
from numpy import linalg as la
from math import *

# Encodes a quantum state: vector of amplitudes of |00>,|01>,|10> and |11>
q = np.array([1,0,0,0])

# CNOT = CX: only if the first bit (MSB) is 1, the second (LSB) is inverted - cx(1,0)
CX = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])

# CZ matrix looks the same regardless of whether MSB or LSB is the control qubit:
CZ = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])

# Hadamard, this is the Kronecker product of the two-dim. matrices H and E (unity)
#H = 1/sqrt(2)*np.array([[1,0,1,0],[0,1,0,1],[1,0,-1,0],[0,1,0,-1]])
H = 1/sqrt(2)*np.kron([[1,1],[1,-1]],[[1,0],[0,1]])

QC1 = np.matmul(CX,H)
QC2 = np.matmul(CZ,QC1)

q1 = np.matmul(QC1,q)
q2 = np.matmul(QC2,q)

print(QC1)
print("\n")
print(QC2)
print("\n")
print(q1)
print(q2)

[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.          0.70710678  0.         -0.70710678]
 [ 0.70710678  0.         -0.70710678  0.        ]]


[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.          0.70710678  0.         -0.70710678]
 [-0.70710678  0.          0.70710678  0.        ]]


[0.70710678 0.         0.         0.70710678]
[ 0.70710678  0.          0.         -0.70710678]


Qiskit's Aer simulator can be used to either simulate a quantum circuit corresponding to some unitary matrix operation and plot the results, or to show the state vectors resulting from the application of certain gates to the QC.

In [77]:
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city

qc = QuantumCircuit(2)
# Prepare |00>, |01>, |10> or |11>:
#qc.x(0)
#qc.x(1)
# Prepare Bell state - e.g. from |00> we'll get a "Phi+"
qc.h(1)
qc.cx(1,0)
# save_statevector should be applied before measurements, we don't want the collapsed post-measurement state
qc.save_statevector(label="1")
qc.x(0)
qc.save_statevector(label="2")
qc.x(0)
qc.save_statevector(label="3")

qc.measure_all()

simulator = Aer.get_backend('aer_simulator')
qc = transpile(qc, simulator)

result = simulator.run(qc).result()
counts = result.get_counts(qc)
#plot_histogram(counts)

data = result.data(0)
data

{'counts': {'0x0': 512, '0x3': 512},
 '2': Statevector([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j,
              0.        +0.j],
             dims=(2, 2)),
 '3': Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
              0.70710678+0.j],
             dims=(2, 2)),
 '1': Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
              0.70710678+0.j],
             dims=(2, 2))}

Another nice utility is the `save_unitary()` method of the Aer simulator that allows to display the matrix corresponding to some circuit. It only works if there are no measurement or reset steps in the QC. You can then compare the results with the matrix multiplications above.

In [79]:
circ = QuantumCircuit(2)
circ.h(1)
circ.cx(1,0)
circ.cz(1,0)
circ.save_unitary()

# Transpile for simulator
simulator = Aer.get_backend('aer_simulator')
circ = transpile(circ, simulator)

# Run and get unitary
result = simulator.run(circ).result()
unitary = result.get_unitary(circ)
print("Circuit unitary:\n", unitary.round(5))

Circuit unitary:
 [[ 0.70711+0.j  0.     +0.j  0.70711-0.j  0.     +0.j]
 [ 0.     +0.j  0.70711+0.j  0.     +0.j  0.70711-0.j]
 [ 0.     +0.j  0.70711+0.j  0.     +0.j -0.70711+0.j]
 [-0.70711+0.j -0.     +0.j  0.70711-0.j -0.     +0.j]]


/tmp/ipykernel_7994/937813038.py:14: DeprecationWarning: The return type of saved unitaries has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Operator` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  print("Circuit unitary:\n", unitary.round(5))
